## Validate Solid angle $\Omega_{AB}$ calculation

### Imports

In [1]:
import sys
sys.path.insert(0, '..')
import pymatcal
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

### Read in the configuration. 
The configuration read-in routine should already been tested by `get_config.ipynb`

In [2]:
config = pymatcal.get_config('configs/pairs-ppdf.yml')

### Get point A
A index is 85

In [3]:
pointA=pymatcal.get_img_voxel_center(85, config['img nvx'], config['mmpvx'])
print('point A:',pointA)

point A: [ 0. 30.  0.]


### Show active detectors

In [4]:
print("Number of active detector units:",config['active det'].shape[0])
print(config['active det'])

Number of active detector units: 2
[[11.   21.   21.   27.   -0.5   0.5   5.    0.48]
 [ 0.   10.    9.   15.   -0.5   0.5   2.    0.48]]


### Setup subdivisions pairs
We pick the first active detector in the test

In [5]:
det_subdivs = pymatcal.get_det_subdivs(
    config['active det'][0], config['det nsub'])
img_subdivs = pymatcal.get_img_subdivs(config['mmpvx'], config['img nsub'])
pAs = img_subdivs['coords']+pointA
pBs = pymatcal.get_centroids(det_subdivs['geoms'])
pAs = pymatcal.coord_transform(
    pymatcal.get_mtransform(config['angle'], -config['dist'], 5), pAs)
abpairs = pymatcal.get_AB_pairs(pAs, pBs)

### Set blocking detector units array and the focused detector unit

In [6]:
blocks = config["det geoms"][config["det geoms"][:,6] != config["active det"][0][6]]
focus_det = np.array([config["active det"][0]])

### Solid angle calculation
$$\Omega_{AB}$$

#### Compare solid angle calculated w/wo subdivision

In [7]:
nvx_total=np.prod(config['img nsub'])
thisdet=config['active det'][0]

1. ##### Without subdivision

In [8]:
new_pointA = pymatcal.coord_transform(pymatcal.get_mtransform(config['angle'], -config['dist'], 5),np.array([pointA]))
AB_pair=np.array([[new_pointA[0][0],new_pointA[0][1],new_pointA[0][2],np.sum(thisdet[0:2])*0.5,np.sum(thisdet[2:4])*0.5,np.sum(thisdet[4:6])*0.5]])
det_incs=np.array([thisdet[1]-thisdet[0],thisdet[3]-thisdet[2],thisdet[5]-thisdet[4]])
sa_terms_whole = pymatcal.get_solid_angles(AB_pair,det_incs)

2. ##### With subdivision

In [9]:
sa_terms = pymatcal.get_solid_angles(abpairs,det_subdivs['incs'])

3. ##### Show the result
There is a roughly 2 fold difference in the sum of the subdivisions solid angles and the whole detector unit solid angle to point A

In [10]:
print("Sum of the subdivisions       :", sa_terms_whole[0])
print("As a whole detector unit to A :", np.sum(sa_terms) / nvx_total)

Sum of the subdivisions       : 0.0013449321329769366
As a whole detector unit to A : 0.0027039814995364728


4. Areas normal to the AB vector

In [11]:
sub_areas= pymatcal.get_norm_areas(abpairs,det_subdivs['incs'])
whole_area = pymatcal.get_norm_areas(AB_pair,det_incs)
print('Sum of the subdivisions       :',np.sum(sub_areas)/nvx_total)
print('As a whole detector unit to A :',np.sum(whole_area))


Sum of the subdivisions       : 14.792937568177601
As a whole detector unit to A : 7.4023825341545
